## Importing Libs. of Python

In [257]:
'''  FOR ML MODELS '''

import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt 


'''  for DL MODELS '''

import tensorflow as tf 
from tensorflow import keras 

## Loading the Required Datasets

In [258]:
train_data  = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [259]:
print("Training Data  has size" , train_data.shape[0] , "rows & " , train_data.shape[1] , "cols")
print("Testing Data  has size" , test_data.shape[0] , "rows & " , test_data.shape[1] , "cols")

Training Data  has size 159256 rows &  24 cols
Testing Data  has size 106171 rows &  23 cols


In [260]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   159256 non-null  int64  
 1   age                  159256 non-null  int64  
 2   height(cm)           159256 non-null  int64  
 3   weight(kg)           159256 non-null  int64  
 4   waist(cm)            159256 non-null  float64
 5   eyesight(left)       159256 non-null  float64
 6   eyesight(right)      159256 non-null  float64
 7   hearing(left)        159256 non-null  int64  
 8   hearing(right)       159256 non-null  int64  
 9   systolic             159256 non-null  int64  
 10  relaxation           159256 non-null  int64  
 11  fasting blood sugar  159256 non-null  int64  
 12  Cholesterol          159256 non-null  int64  
 13  triglyceride         159256 non-null  int64  
 14  HDL                  159256 non-null  int64  
 15  LDL              

In [261]:
train_data.isnull().sum()

id                     0
age                    0
height(cm)             0
weight(kg)             0
waist(cm)              0
eyesight(left)         0
eyesight(right)        0
hearing(left)          0
hearing(right)         0
systolic               0
relaxation             0
fasting blood sugar    0
Cholesterol            0
triglyceride           0
HDL                    0
LDL                    0
hemoglobin             0
Urine protein          0
serum creatinine       0
AST                    0
ALT                    0
Gtp                    0
dental caries          0
smoking                0
dtype: int64

In [262]:
train_data.__dict__

{'_is_copy': None,
 '_mgr': BlockManager
 Items: Index(['id', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)',
        'eyesight(right)', 'hearing(left)', 'hearing(right)', 'systolic',
        'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride',
        'HDL', 'LDL', 'hemoglobin', 'Urine protein', 'serum creatinine', 'AST',
        'ALT', 'Gtp', 'dental caries', 'smoking'],
       dtype='object')
 Axis 1: RangeIndex(start=0, stop=159256, step=1)
 NumericBlock: [ 4  5  6 16 18], 5 x 159256, dtype: float64
 NumericBlock: [ 0  1  2  3  7  8  9 10 11 12 13 14 15 17 19 20 21 22 23], 19 x 159256, dtype: int64,
 '_item_cache': {'id': 0              0
  1              1
  2              2
  3              3
  4              4
             ...  
  159251    159251
  159252    159252
  159253    159253
  159254    159254
  159255    159255
  Name: id, Length: 159256, dtype: int64,
  'age': 0         55
  1         70
  2         20
  3         35
  4         30
  

In [263]:
col = test_data['id']

In [264]:
''' Removing ID column from dataframe '''
train_data = train_data .drop(['id'] , axis =1)
test_data = test_data.drop(['id'] , axis =1 )

In [265]:
X_train = train_data.iloc[: , :22]
y_train = train_data['smoking']

X_test = test_data.iloc[: , :23]
y_train

0         1
1         0
2         1
3         0
4         1
         ..
159251    0
159252    0
159253    0
159254    1
159255    0
Name: smoking, Length: 159256, dtype: int64

In [266]:
from sklearn.preprocessing import StandardScaler 

scale1 = StandardScaler()


X_train[['height(cm)', 'weight(kg)', 'waist(cm)' , 'fasting blood sugar', 'Cholesterol', 'triglyceride']] = scale1.fit_transform(X_train[['height(cm)' ,'weight(kg)', 'waist(cm)' , 'fasting blood sugar', 'Cholesterol', 'triglyceride']] )
X_train



,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,55,-0.030268,-0.567581,-0.223489,0.5,0.6,1,1,135,87,...,2.604429,40,75,16.5,1,1.0,22,25,27,0
1,70,-0.030268,-0.170319,0.669577,0.6,0.7,2,2,146,83,...,-1.097105,57,126,16.2,1,1.1,27,23,37,1
2,20,0.536694,0.624205,-0.223489,0.4,0.5,1,1,118,75,...,1.048274,45,93,17.4,1,0.8,27,31,53,0
3,35,1.670617,2.213252,2.455708,1.5,1.2,1,1,131,88,...,1.138923,38,102,15.9,1,1.0,20,27,30,1
4,30,-0.030268,-0.567581,-0.279305,1.5,1.0,1,1,121,76,...,-0.613639,44,93,15.4,1,0.8,19,13,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,40,-1.164191,-1.759366,-1.563087,1.5,2.0,1,1,127,80,...,-1.217971,72,159,14.5,1,0.8,25,26,13,0
159252,50,-1.164191,0.624205,-0.111855,1.0,1.0,1,1,120,80,...,1.123815,64,108,14.5,1,0.6,21,20,18,0
159253,40,-0.597229,-1.362104,-1.897986,1.5,1.0,1,1,114,70,...,-1.248188,87,93,10.9,1,0.6,15,9,12,0
159254,50,-0.030268,0.624205,1.004477,1.2,1.0,1,1,121,90,...,0.307967,55,80,14.4,1,1.1,22,17,37,0


In [267]:
y_train

0         1
1         0
2         1
3         0
4         1
         ..
159251    0
159252    0
159253    0
159254    1
159255    0
Name: smoking, Length: 159256, dtype: int64

## Approach 1 - LOGISTIC REGRESSION 

In [268]:
from sklearn.linear_model import LogisticRegression 

lr = LogisticRegression()

lr.fit(X_train , y_train )

C:\Users\HP\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [269]:
prediction = lr.predict(X_test)
#np.set_printoptions(threshold=np.inf)
# prediction

In [271]:
col.shape
# prediction = prediction.reshape((-1 , 1))
prediction.shape


(106171,)

In [273]:
col = np.array(col)
prediction = np.array(prediction)

col = col.reshape(-1 , 1)
prediction  = prediction.reshape(-1 , 1)

sol = np.concatenate((col , prediction) , axis=1  )

sol = pd.DataFrame(sol , columns  =['id' , 'smoking'])
res = sol.to_csv("solution.csv")
